#### kompresja SVD, tworzenie macierzy topologii trójwymiarowej

In [2]:
import numpy as np
import scipy.sparse as sp
from sklearn.utils.extmath import randomized_svd
import matplotlib.pyplot as plt


def generate_matrix(k):
    n = 2 ** (3 * k)
    
    matrix = np.zeros((n, n))
    
    row_length = 2 ** k
    neighbours = [-1, 1, row_length, -row_length, row_length ** 2, -(row_length ** 2)]
    
    for i in range(n):
        for neigh in neighbours:
            neight_id = i + neigh
            matrix[i, i] = 1
            if neight_id < 0 or neight_id >= n: 
                continue
                
            # Same row
            if neight_id // row_length == i // row_length:
                matrix[i, neight_id] = 1
            
            # Same column
            elif neight_id % row_length == i % row_length:
                matrix[i, neight_id] = 1
            
            # Same in third dimension
            else:
                matrix[i, neight_id] = 1
    

    return matrix


class CompressNode:
    def __init__(self, rank=0, children=None, singular_values=None, U=None, V=None, size=None):
        self.rank = rank
        self.children = children
        self.s = singular_values
        self.U = U
        self.V = V
        self.size = size
        self.val = None
        
    def append_child(self, child):
        self.children.append(child)
    
    def __str__(self):
        return f'Node: rank = {self.rank}; children: {self.children}; s: {self.s}; U = {self.U}; V = {self.V}, size = {self.size}'
    

def compress_matrix(matrix, U, s, V, r):
    node = None
    if (matrix == 0).all():
        node = CompressNode(rank=0, size=matrix.shape)
    
    else:
        D = np.diag(s)
        node = CompressNode(rank=r, singular_values=s, U=U, V=D @ V, 
                            size=matrix.shape)
    
    return node

def create_tree(matrix, r, epsilon):
    U, s, V = randomized_svd(matrix, n_components=r)
    rank = s.size
    node = None
    if s[-1] < epsilon:
        node = compress_matrix(matrix, U, s, V, rank)
    
    else:
        Y, X = matrix.shape
        
        if Y == X == 1:
            node = CompressNode(rank=1, size=(1, 1))
            node.val = matrix
            return node
        
        node = CompressNode(rank=r, children=[], size=(Y, X))
        
        # 4 childrens
        
        node.append_child(create_tree(matrix[0: Y // 2, 0: X // 2], r, epsilon))
        node.append_child(create_tree(matrix[0: Y // 2, X // 2: X], r, epsilon))
        node.append_child(create_tree(matrix[Y // 2: Y, 0: X // 2], r, epsilon))
        node.append_child(create_tree(matrix[Y // 2: Y, X // 2: X], r, epsilon))
        
    return node


def _canvas_matrix(wisualized, root, x, y, n): #n must be 2^i where i is natural number
    
    
    if root.U is None and root.V is None and root.rank == 0: # Only zeros
        return
    
    elif root.children is None: # Leaf
        if root.val is not None: # Leaf
            wisualized[y, x] = 128 # Gray
            return
        else:
            xe = x + root.size[1]
            ye = y + root.size[0]

            wisualized[y: y + root.rank, x: xe] = np.ones((root.rank, xe - x)) * 64  # Horizontal
            wisualized[y: ye, x: x + root.rank] = np.ones((ye - y, root.rank)) * 64  # Vertical
            
            return
    
    else:
        childs = root.children
        n = n//2
        _canvas_matrix(wisualized, childs[0], x, y, n)
        _canvas_matrix(wisualized, childs[1], x + n, y, n)
        _canvas_matrix(wisualized, childs[2], x, y + n, n)
        _canvas_matrix(wisualized, childs[3], x + n, y + n, n)
        return

def wisualize_svd(matrix_tree: CompressNode, n):
    wisualized = np.ones((n, n)) * 255
    _canvas_matrix(wisualized, matrix_tree, 0, 0, n)
    plt.imshow(wisualized, 'gray')

def decompression(root):
    if root.U is None and root.V is None and root.rank == 0: # Only zeros
        return np.zeros(root.size)
    
    elif root.children is None: # Leaf
        if root.val is not None: # Leaf
            return root.val
        return root.U @ root.V # maybe @singular_values, but I think it has already done
    
    else:
        children_dec = [decompression(child) for child in root.children]
        
        
        return np.vstack((np.hstack((children_dec[0], children_dec[1])),
                        np.hstack((children_dec[2], children_dec[3]))))    

def diff(original, decompressed):
    return np.sum(np.square(original - decompressed))

## Mnożenie skompresowanej macierzy przez wektor

In [111]:
def mul_vek(copmresed_matrix: CompressNode, vector : np.array):
    if copmresed_matrix.U is None and copmresed_matrix.V is None and copmresed_matrix.rank == 0: # Only zeros
        return np.zeros(vector.size)
    
    elif copmresed_matrix.children is None: # Leaf
        if copmresed_matrix.val is not None: # Leaf
            return copmresed_matrix.val @ vector.T
        return (copmresed_matrix.U @ (copmresed_matrix.V @ vector.T)).T
    else:
        head = vector[:(len(vector)//2)]
        tail = vector[(len(vector)//2):]
        
        children = copmresed_matrix.children
        result_head = mul_vek(children[0], head) + mul_vek(children[1], tail)
        result_tail = mul_vek(children[2], head) + mul_vek(children[3], tail)
        return np.concatenate((result_head, result_tail))
    

Sprawdzenie poprawności

In [112]:
M = generate_matrix(3)
x = np.ones(len(M))
len(x)
normal_mul = M @ x.T
my_mul = mul_vek(create_tree(M, 2, 0.00001), x)

print(sum((normal_mul-my_mul)**2))

# it's ok

1.0097419586828951e-27


## Mnożenie macierzy skompresowanych

In [116]:
def _sum_compresed_matrix(mat1: CompressNode, mat2: CompressNode, current_node : CompressNode):
    pass

def _mul_compresed_matrix(mat1: CompressNode, mat2: CompressNode, current_node : CompressNode):

    if (mat1.U is None and mat1.V is None and mat1.rank == 0) or (mat2.U is None and mat2.V is None and mat2.rank == 0): # one matrix only zeros
        current_node.U = None
        current_node.V = None
        current_node.rank = 0
        return current_node
    elif mat1.children is None and mat2.children is None: # same dim compresion
        if mat1.val is not None and mat2.val is not None:
            current_node.val = mat1.val @ mat2.val
            return current_node
        elif mat1.val is not None:
            current_node.U = mat2.U @ mat1.val
            current_node.V = mat2.V
            current_node.rank = mat2.rank
            return current_node
        elif mat2.val is not None:
            current_node.U = mat1.U
            current_node.V = mat2.val @ mat1.V
            current_node.rank = mat1.rank
            return current_node
        else: #both matrixes are U@V
            current_node.U = mat1.U @ (mat1.V @ mat2.U)
            current_node.V = mat2.V
            current_node.rank = max(mat1.rank, mat2.rank)
            return current_node
    elif mat1.children is None:
        U = mat1.U
        V = mat1.V
        r = mat1.rank

        n = U.shape[1] // 2
        m = V.shape[0] // 2

        U_head = U[:, :n]
        U_tail = U[:, n:]

        V_head = V[:m, :]
        V_tail = V[m:, :]
        
        A11, A12, A21, A22 = CompressNode(r, U=U_head, V=V_head), CompressNode(r, U=U_head, V=V_tail), CompressNode(r, U=U_tail, V=V_head), CompressNode(r, U=U_tail, V=V_tail)

        current_node.append_child(_sum_compresed_matrix(
            _mul_compresed_matrix(A11, mat2.children[0], CompressNode())
            , _mul_compresed_matrix(A12, mat2.children[2], CompressNode())
            , CompressNode()
            ))
        current_node.append_child(_sum_compresed_matrix(
            _mul_compresed_matrix(A11, mat2.children[1], CompressNode())
            , _mul_compresed_matrix(A12, mat2.children[3], CompressNode())
            , CompressNode()))
        current_node.append_child(_sum_compresed_matrix(
            _mul_compresed_matrix(A21, mat2.children[0], CompressNode())
            , _mul_compresed_matrix(A22, mat2.children[2], CompressNode())
            , CompressNode()))
        current_node.append_child(_sum_compresed_matrix(
            _mul_compresed_matrix(A21, mat2.children[1], CompressNode())
            , _mul_compresed_matrix(A22, mat2.children[3], CompressNode())
            , CompressNode()))
        
        return current_node


    elif mat2.children is None:
        U = mat2.U
        V = mat2.V
        r = mat2.rank

        n = U.shape[1] // 2
        m = V.shape[0] // 2

        U_head = U[:, :n]
        U_tail = U[:, n:]

        V_head = V[:m, :]
        V_tail = V[m:, :]

        B11, B12, B21, B22 = CompressNode(r, U=U_head, V=V_head), CompressNode(r, U=U_head, V=V_tail), CompressNode(r, U=U_tail, V=V_head), CompressNode(r, U=U_tail, V=V_tail)

        current_node.append_child(_sum_compresed_matrix(
            _mul_compresed_matrix(mat1.children[0], B11, CompressNode())
            , _mul_compresed_matrix(mat1.children[1], B21, CompressNode())
            , CompressNode()
            ))
        current_node.append_child(_sum_compresed_matrix(
            _mul_compresed_matrix(mat1.children[0], B12, CompressNode())
            , _mul_compresed_matrix(mat1.children[1], B22, CompressNode())
            , CompressNode()))
        current_node.append_child(_sum_compresed_matrix(
            _mul_compresed_matrix(mat1.children[2], B11, CompressNode())
            , _mul_compresed_matrix(mat1.children[3], B21, CompressNode())
            , CompressNode()))
        current_node.append_child(_sum_compresed_matrix(
            _mul_compresed_matrix(mat1.children[2], B12, CompressNode())
            , _mul_compresed_matrix(mat1.children[3], B22, CompressNode())
            , CompressNode()))
        
        return current_node
    
    else:
        current_node.append_child(_sum_compresed_matrix(
            _mul_compresed_matrix(mat1.children[0], mat2.children[0], CompressNode())
            , _mul_compresed_matrix(mat1.children[1], mat2.children[2], CompressNode())
            , CompressNode()
            ))
        current_node.append_child(_sum_compresed_matrix(
            _mul_compresed_matrix(mat1.children[0], mat2.children[1], CompressNode())
            , _mul_compresed_matrix(mat1.children[1], mat2.children[3], CompressNode())
            , CompressNode()))
        current_node.append_child(_sum_compresed_matrix(
            _mul_compresed_matrix(mat1.children[2], mat2.children[0], CompressNode())
            , _mul_compresed_matrix(mat1.children[3], mat2.children[2], CompressNode())
            , CompressNode()))
        current_node.append_child(_sum_compresed_matrix(
            _mul_compresed_matrix(mat1.children[2], mat2.children[1], CompressNode())
            , _mul_compresed_matrix(mat1.children[3], mat2.children[3], CompressNode())
            , CompressNode()))
        
        return current_node
        

def mul_compresed_matrix(mat1: CompressNode, mat2: CompressNode):
    root = CompressNode(rank = max(mat1.rank, mat2.rank))
    _mul_compresed_matrix(mat1, mat2, root)
    return root


In [115]:
import numpy as np

# Przykładowe macierze U i V
U = np.array([[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9],
              [10, 11, 12]])

V = np.array([[13, 14, 15],
              [16, 17, 18],
              [19, 20, 21],
              [22, 23, 24]])

# Podział macierzy na połowę względem wiersza
n = U.shape[0] // 2
m = V.shape[0] // 2

U_head = U[:n, :]
U_tail = U[n:, :]

V_head = V[:m, :]
V_tail = V[m:, :]

print("U_head:")
print(U_head)

print("\nU_tail:")
print(U_tail)

print("\nV_head:")
print(V_head)

print("\nV_tail:")
print(V_tail)

U_head:
[[ 1]
 [ 4]
 [ 7]
 [10]]

U_tail:
[[ 2  3]
 [ 5  6]
 [ 8  9]
 [11 12]]

V_head:
[[13]
 [16]
 [19]
 [22]]

V_tail:
[[14 15]
 [17 18]
 [20 21]
 [23 24]]
